In [203]:
import pandas as pd
from glob import glob

MAIN_SOURCE = True

LIVE_PRED_START = pd.Timestamp.today() + pd.to_timedelta(1, "d")
LIVE_PRED_START = LIVE_PRED_START.strftime("%Y-%m-%d")

LIVE_PRED_END = pd.to_datetime(LIVE_PRED_START) + pd.to_timedelta(1, "d")
LIVE_PRED_END = LIVE_PRED_END.date().strftime("%Y-%m-%d")

START_HOUR = 9
END_HOUR = 18

print("Pred Date Start: ", LIVE_PRED_START, "Pred Date End: ", LIVE_PRED_END)

stock_price_files = glob("../Dataset/*")

stock_price_list = [pd.read_csv(v) for v in stock_price_files]
stock_price_df = pd.concat(stock_price_list, ignore_index=True)
stock_price_df["timestamp"] = pd.to_datetime(
    stock_price_df.timestamp).dt.tz_localize(None)
stock_price_df.sort_values(
    by=["short_name", "timestamp"], ignore_index=True, inplace=True
)
stock_price_df.drop_duplicates(ignore_index=True, inplace=True)
stock_price_df["date"] = stock_price_df.timestamp.dt.date
stock_price_df["hour"] = stock_price_df.timestamp.dt.hour

stock_names = list(stock_price_df.short_name.unique())

processed_data = pd.read_csv("./stock_price_processed_data_live.csv.gz")
processed_data.head()

Pred Date Start:  2024-01-02 Pred Date End:  2024-01-03


,timestamp,date,hour,AKBNK,ARCLK,ASELS,BIMAS,DOHOL,EKGYO,EREGL,...,LAG_5DAY_YKBNK,year,month,day,dow,quarter,doy,woy,is_monday_morning,is_friday_noon
0,2018-01-02 09:00:00,2018-01-02,9,0.016222,0.004632,0.001813,0.010882,0.011668,0.010630,0.002966,...,NaN,2018,1,2,1,1,2,1,0,0
1,2018-01-02 10:00:00,2018-01-02,10,0.004986,-0.006459,-0.001810,-0.003807,0.000000,-0.003463,-0.003937,...,NaN,2018,1,2,1,1,2,1,0,0
2,2018-01-02 11:00:00,2018-01-02,11,-0.001973,-0.000930,0.003640,-0.005713,0.000000,0.003475,0.000000,...,NaN,2018,1,2,1,1,2,1,0,0
3,2018-01-02 12:00:00,2018-01-02,12,0.002966,0.000000,0.000602,0.004470,0.011111,0.000000,0.001968,...,NaN,2018,1,2,1,1,2,1,0,0
4,2018-01-02 13:00:00,2018-01-02,13,0.004970,0.005582,-0.001204,0.004456,-0.010989,0.003463,0.003945,...,NaN,2018,1,2,1,1,2,1,0,0


In [204]:
TEST_START = LIVE_PRED_START

drop_cols = ["timestamp", "date"] + stock_names

train_data = processed_data[processed_data.date < TEST_START].reset_index(drop=True)
test_data = processed_data[processed_data.date >= TEST_START].reset_index(drop=True)

X_train = train_data.drop(columns=drop_cols)
y_train = train_data[stock_names].copy()
X_test = test_data.drop(columns=drop_cols)

X_test.tail()

,hour,AKBNK_CLOSE,AKBNK_DIVIDENDS,AKBNK_HIGH,AKBNK_LOW,AKBNK_OPEN,AKBNK_STOCKSPLITS,AKBNK_VOLUME,ARCLK_CLOSE,ARCLK_DIVIDENDS,...,LAG_5DAY_YKBNK,year,month,day,dow,quarter,doy,woy,is_monday_morning,is_friday_noon
5,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.87,2024,1,2,1,1,2,1,0,0
6,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.89,2024,1,2,1,1,2,1,0,0
7,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.84,2024,1,2,1,1,2,1,0,0
8,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.00,2024,1,2,1,1,2,1,0,0
9,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.06,2024,1,2,1,1,2,1,0,0


In [17]:
import catboost as cb
READ_MODEL_FIT = True

if READ_MODEL_FIT:
    cbregr = cb.CatBoostRegressor()
    cbregr.load_model("./catboost_fit")
else:
    cbregr = cb.CatBoostRegressor(objective="MultiRMSE", random_seed=6969)
    cbregr.fit(X_train, y_train)
    cbregr.save_model("./catboost_fit")
     
cb_preds = cbregr.predict(X_test)
cb_preds

0:	learn: 0.0483117	total: 2.81s	remaining: 46m 51s
1:	learn: 0.0482845	total: 5.34s	remaining: 44m 24s
2:	learn: 0.0482628	total: 7.89s	remaining: 43m 41s
3:	learn: 0.0482434	total: 10.4s	remaining: 43m 12s
4:	learn: 0.0482193	total: 12.9s	remaining: 42m 56s
5:	learn: 0.0481986	total: 15.5s	remaining: 42m 43s
6:	learn: 0.0481806	total: 18.1s	remaining: 42m 42s
7:	learn: 0.0481529	total: 20.6s	remaining: 42m 33s
8:	learn: 0.0481306	total: 23.1s	remaining: 42m 27s
9:	learn: 0.0481140	total: 25.7s	remaining: 42m 27s
10:	learn: 0.0481065	total: 28.4s	remaining: 42m 33s
11:	learn: 0.0480982	total: 30.9s	remaining: 42m 27s
12:	learn: 0.0480820	total: 33.5s	remaining: 42m 20s
13:	learn: 0.0480628	total: 36.1s	remaining: 42m 22s
14:	learn: 0.0480477	total: 38.7s	remaining: 42m 18s
15:	learn: 0.0480337	total: 41.3s	remaining: 42m 17s
16:	learn: 0.0480232	total: 44.2s	remaining: 42m 35s
17:	learn: 0.0480046	total: 47.2s	remaining: 42m 52s
18:	learn: 0.0479917	total: 50.1s	remaining: 43m 7s
19:	

array([[ 1.94571449e-03, -1.18783167e-03,  2.88493443e-03, ...,
         7.45234466e-04,  1.12227752e-03,  3.02184092e-03],
       [ 1.13450686e-03, -6.94243227e-04,  1.18361892e-03, ...,
        -1.52150010e-03,  7.89900212e-04,  1.54923355e-03],
       [ 6.77917976e-04, -1.02518363e-03,  3.47151907e-04, ...,
        -1.74631356e-03, -1.78199744e-04,  9.49722648e-04],
       ...,
       [ 6.95107142e-05,  3.49211003e-04,  7.73537655e-04, ...,
        -6.16256684e-04, -3.66806028e-05,  9.39091114e-05],
       [ 5.66340494e-04,  8.26692117e-04,  1.21008715e-03, ...,
        -3.69361212e-04,  2.67828861e-04,  4.57134587e-04],
       [ 5.33546937e-03,  4.88980719e-03,  3.79075696e-03, ...,
         5.68885487e-03,  4.37082419e-03,  4.18664322e-03]])

In [51]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

lgbm_model = lgb.LGBMRegressor(random_state=6969, objective="regression_l1", importance_type="gain")
model = MultiOutputRegressor(lgbm_model, n_jobs=-1)
model.fit(X_train, y_train)
lgb_preds = model.predict(X_test)

In [136]:
import optuna

VALID_START = "2023-01-01"

tuning_train = processed_data[processed_data.date < VALID_START].reset_index(drop=True)
tuning_valid = processed_data[(processed_data.date >= VALID_START) & (processed_data.date < TEST_START)].reset_index(drop=True)

def pred_to_result_dataframe(preds, close_hour = 18):

    result = test_data[["date", "hour"]].copy()
    pred_df = pd.DataFrame(preds, columns=stock_names)
    result = pd.concat([result, pred_df], axis=1)
    result = result.melt(id_vars=["date", "hour"], value_name="prediction_return", var_name="short_name")
    result["date"] = pd.to_datetime(result.date)

    stock_price_df["date"] = pd.to_datetime(stock_price_df.date)
    yesterday_closes = stock_price_df[stock_price_df["hour"] == close_hour].reset_index(drop=True)
    result = result.merge(yesterday_closes[["date", "hour", "short_name", "price"]], how="left")
    result["price"] = result["price"].bfill()
    result.rename(columns={"price": "yesterday_close"}, inplace=True)
    result["yesterday_close"] = result["yesterday_close"].shift(10)
    result["prediction_price"] = result.yesterday_close * (1 + result.prediction_return)

    result = result.merge(stock_price_df[["date", "hour", "short_name", "price"]], how="left").dropna(axis=0)
    
    return result

def objective(trial):

    x_tuning_train = tuning_train.drop(columns=drop_cols)
    y_tuning_train = tuning_train[stock_names].copy()
    x_tuning_valid = tuning_valid.drop(columns=drop_cols)

    param = {
        "objective": trial.suggest_categorical("objective", ["regression", "regression_l1", "mape"]),
        "metric": "mae",
        "verbosity": -1,
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart", "rf"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": 1,
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_depth": trial.suggest_int("max_depth", 5, 8),
        "feature_pre_filter": False,
        "num_iterations": trial.suggest_int("num_iterations", 100, 750),
        "boost_from_average": trial.suggest_categorical("boost_from_average", [True, False]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.15),
    }

    model = MultiOutputRegressor(lgb.LGBMRegressor(**param), n_jobs=-1)
    model.fit(x_tuning_train, y_tuning_train)
    preds = model.predict(x_tuning_valid)

    tuning_result = pred_to_result_dataframe(preds)

    wmape = tuning_result.groupby("short_name").apply(lambda x: pd.Series({
        "wmape": abs(x["prediction_price"] - x["price"]).sum() / x["price"].sum()
        })).wmape.mean()
    
    return wmape

study = optuna.create_study(direction="minimize") # timeout=5*3600
study.optimize(objective, timeout=3*3600)
study.trials_dataframe().to_csv("./study_dataframe_optuna_lgbm.csv", index=False)

[I 2023-12-31 20:39:29,019] A new study created in memory with name: no-name-56f1a27b-caca-4ad2-88f2-10bf355090d2


[LightGBM] [Warning] bagging_fraction is set=0.7661505048284494, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7661505048284494
[LightGBM] [Warning] lambda_l1 is set=8.116760233134114, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.116760233134114
[LightGBM] [Warning] feature_fraction is set=0.9235832706586538, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9235832706586538
[LightGBM] [Warning] lambda_l2 is set=2.1596402188879506e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.1596402188879506e-08
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.7661505048284494, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7661505048284494
[LightGBM] [Warning] lambda_l1 is set=8.116760233134114, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.116760233134114
[LightGBM] [Warning] feature_fraction is

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
[I 2023-12-31 20:43:47,431] Trial 0 finished with value: 0.016091326353850507 and parameters: {'objective': 'mape', 'boosting_type': 'dart', 'lambda_l1': 8.116760233134114, 'lambda_l2': 2.1596402188879506e-08, 'num_leaves': 222, 'feature_fraction': 0.9235832706586538, 'bagging_fraction': 0.7661505048284494, 'min_child_samples': 63, 'max_depth': 8, 'num_iterations': 239, 'boost_from_average': True, 'learning_rate': 0.017451165783280074}. Best is trial 0 with value: 0.016091326353850507.


[LightGBM] [Warning] bagging_fraction is set=0.7933144813085115, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7933144813085115
[LightGBM] [Warning] lambda_l1 is set=0.0004027326420035557, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004027326420035557
[LightGBM] [Warning] feature_fraction is set=0.8109262163052546, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8109262163052546
[LightGBM] [Warning] lambda_l2 is set=0.4287647758582228, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.4287647758582228
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.7933144813085115, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7933144813085115
[LightGBM] [Warning] lambda_l1 is set=0.0004027326420035557, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004027326420035557
[LightGBM] [Warning] feature_fra

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
[I 2023-12-31 20:46:37,696] Trial 1 finished with value: 0.016830721250473118 and parameters: {'objective': 'regression', 'boosting_type': 'gbdt', 'lambda_l1': 0.0004027326420035557, 'lambda_l2': 0.4287647758582228, 'num_leaves': 52, 'feature_fraction': 0.8109262163052546, 'bagging_fraction': 0.7933144813085115, 'min_child_samples': 44, 'max_depth': 5, 'num_iterations': 474, 'boost_from_average': True, 'learning_rate': 0.07927335733064181}. Best is trial 0 with value: 0.016091326353850507.


...

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
[I 2023-12-31 23:35:56,548] Trial 83 finished with value: 0.01607264025061446 and parameters: {'objective': 'regression', 'boosting_type': 'rf', 'lambda_l1': 6.874511937470793, 'lambda_l2': 3.4467962431046606e-07, 'num_leaves': 132, 'feature_fraction': 0.9082832027167113, 'bagging_fraction': 0.7350772752602812, 'min_child_samples': 13, 'max_depth': 6, 'num_iterations': 273, 'boost_from_average': True, 'learning_rate': 0.04996908657788224}. Best is trial 10 with value: 0.01607264025061446.


[LightGBM] [Warning] bagging_fraction is set=0.5066265234369278, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5066265234369278
[LightGBM] [Warning] lambda_l1 is set=0.0012935549284730666, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0012935549284730666
[LightGBM] [Warning] feature_fraction is set=0.9142396682228053, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9142396682228053
[LightGBM] [Warning] lambda_l2 is set=5.447923993284767e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.447923993284767e-08
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.5066265234369278, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5066265234369278
[LightGBM] [Warning] lambda_l1 is set=0.0012935549284730666, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0012935549284730666
[LightGBM] [Warning] featu

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
[I 2023-12-31 23:38:39,021] Trial 84 finished with value: 0.016315187597222867 and parameters: {'objective': 'regression', 'boosting_type': 'rf', 'lambda_l1': 0.0012935549284730666, 'lambda_l2': 5.447923993284767e-08, 'num_leaves': 108, 'feature_fraction': 0.9142396682228053, 'bagging_fraction': 0.5066265234369278, 'min_child_samples': 96, 'max_depth': 6, 'num_iterations': 216, 'boost_from_average': True, 'learning_rate': 0.05593973410660888}. Best is trial 10 with value: 0.01607264025061446.


[LightGBM] [Warning] bagging_fraction is set=0.4200272318501896, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4200272318501896
[LightGBM] [Warning] lambda_l1 is set=9.041712133611075, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.041712133611075
[LightGBM] [Warning] feature_fraction is set=0.9797519233897504, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9797519233897504
[LightGBM] [Warning] lambda_l2 is set=4.076317018467136e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.076317018467136e-07
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.4200272318501896, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4200272318501896
[LightGBM] [Warning] lambda_l1 is set=9.041712133611075, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.041712133611075
[LightGBM] [Warning] feature_fraction is s

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1140: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
[I 2023-12-31 23:39:34,339] Trial 85 finished with value: 0.01607264025061446 and parameters: {'objective': 'regression', 'boosting_type': 'rf', 'lambda_l1': 9.041712133611075, 'lambda_l2': 4.076317018467136e-07, 'num_leaves': 150, 'feature_fraction': 0.9797519233897504, 'bagging_fraction': 0.4200272318501896, 'min_child_samples': 28, 'max_depth': 7, 'num_iterations': 257, 'boost_from_average': True, 'learning_rate': 0.024549571773505325}. Best is trial 10 with value: 0.01607264025061446.


In [202]:
best_param

{'objective': 'regression',
 'boosting_type': 'rf',
 'lambda_l1': 8.816979942542755,
 'lambda_l2': 3.281542231961584e-06,
 'num_leaves': 138,
 'feature_fraction': 0.9948710619824198,
 'bagging_fraction': 0.43847967094014917,
 'min_child_samples': 92,
 'max_depth': 5,
 'num_iterations': 114,
 'boost_from_average': True,
 'learning_rate': 0.04548612752166902,
 'bagging_freq': 1}

In [197]:
best_param = study.best_params
best_param["bagging_freq"] = 1
# best_param["feature_pre_filter"] = False

lgb_tuned = MultiOutputRegressor(lgb.LGBMRegressor(**best_param), n_jobs=-1)
lgb_tuned.fit(X_train, y_train)
lgb_preds_tuned = lgb_tuned.predict(X_test)

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(

[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction is set=0.9948710619824198, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9948710619824198
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=3.281542231961584e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.281542231961584e-06
[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction 

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction is set=0.9948710619824198, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9948710619824198
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=3.281542231961584e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.281542231961584e-06
[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction 

/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction is set=0.9948710619824198, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9948710619824198
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=3.281542231961584e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.281542231961584e-06


/home/ssc/miniconda3/envs/foreflux/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction is set=0.9948710619824198, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9948710619824198
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=3.281542231961584e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.281542231961584e-06
[LightGBM] [Warning] bagging_fraction is set=0.43847967094014917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43847967094014917
[LightGBM] [Warning] lambda_l1 is set=8.816979942542755, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.816979942542755
[LightGBM] [Warning] feature_fraction 

In [180]:
def raw_result_to_daily_errors(result_df, live_start_date = "2023-12-25"):
    
    errors = result_df.groupby(["date", "short_name"]).apply(
        lambda x: pd.Series({
            "wmape": abs(x["prediction_price"] - x["price"]).sum() / x["price"].sum()
            })
            ).reset_index()
    errors = errors[errors.date >= live_start_date].reset_index(drop=True)

    return errors.groupby(["date"]).wmape.mean()

def raw_result_to_ensemble_result(ensemble_dictionary):

    ensemble_df = pd.concat(ensemble_dictionary).reset_index().drop(columns=["level_1"])
    ensemble_df.rename(columns={"level_0": "type"}, inplace=True)
    ensemble_df = ensemble_df.groupby(
        ["date", "hour", "short_name", "yesterday_close"]
        )[["price", "prediction_return", "prediction_price"]].mean().reset_index()
    
    ensemble_dictionary["ensemble"] = ensemble_df
    
    return ensemble_df[list(catboost_result)]

catboost_result = pred_to_result_dataframe(cb_preds)
lightgbm_result = pred_to_result_dataframe(lgb_preds)
lightgbm_tuned = pred_to_result_dataframe(lgb_preds_tuned)

ensemble_dict = {
    "catboost-default": catboost_result,
    "lightgbm-default": lightgbm_result,
    "lightgbm-tuned": lightgbm_tuned
}
ensemble_result = raw_result_to_ensemble_result(ensemble_dict)

print(raw_result_to_daily_errors(lightgbm_result))
print(raw_result_to_daily_errors(lightgbm_tuned))
print(raw_result_to_daily_errors(catboost_result))
print(raw_result_to_daily_errors(ensemble_result))

date
2023-12-25    0.027892
2023-12-26    0.014122
2023-12-27    0.016455
2023-12-28    0.021119
Name: wmape, dtype: float64
date
2023-12-25    0.028508
2023-12-26    0.013953
2023-12-27    0.016339
2023-12-28    0.020509
Name: wmape, dtype: float64
date
2023-12-25    0.031589
2023-12-26    0.013977
2023-12-27    0.016765
2023-12-28    0.019673
Name: wmape, dtype: float64
date
2023-12-25    0.028224
2023-12-26    0.014143
2023-12-27    0.016364
2023-12-28    0.021351
Name: wmape, dtype: float64
